In [8]:
import pandas as pd

In [9]:
def sell_dataframe_change(df_sell):
    df_sell['sku_num'] = df_sell['sku_num'].apply(lambda x: 1 if x.find('ap') == 6 else 0 )
    df_sell['pen'] = df_sell['sku_num'].apply(lambda x:1 if x == 0 else 0 )
    df_sell['apple'] = df_sell['sku_num'] 
    df_sell.drop(['sku_num'], axis = 1, inplace = True)
    df_sell = df_sell.groupby(['date'])['apple', 'pen'].sum()
    df_sell['date'] = df_sell.index
    df_sell = df_sell[['date','apple','pen']]
    df_sell=df_sell.reset_index(drop=True)
    return df_sell
#df_sell.head()

In [10]:
def daily_inv( df_sell, df_supply):
    df_daily_inv = pd.DataFrame(columns=['date','apple','pen'])
    df_daily_inv['date'] = df_sell['date']
    df_daily_inv['apple'] = - df_sell['apple']
    df_daily_inv['pen'] = - df_sell['pen']
    df_daily_inv = pd.concat([df_daily_inv,df_supply])
    df_daily_inv = df_daily_inv.groupby(['date']).sum()
    df_daily_inv['apple'] = df_daily_inv['apple'].cumsum()
    df_daily_inv['pen'] = df_daily_inv['pen'].cumsum()
    df_daily_inv = df_daily_inv.reset_index()
    return df_daily_inv
#df_daily_inv.head()

In [11]:
def month_steal(df_inv, df_daily_inv):
    steal = pd.DataFrame(columns = ['ap','pe'])
    steal['ap'] = -df_inv['apple']
    steal['pe'] = -df_inv['pen']
    df_daily_inv.index = df_daily_inv.date
    steal.index = df_inv.date
    df_daily_inv.drop(['date'], axis = 1, inplace = True)
    steal = pd.concat([steal, df_daily_inv], axis = 1, join = 'inner')
    steal['apple'] = steal['apple']+steal['ap']
    steal['pen'] = steal['pen']+steal['pe']
    steal.drop(['ap', 'pe'], axis = 1, inplace = True)
    steal['apple'] = steal['apple']-steal['apple'].shift(1).fillna(0)
    steal['pen'] = steal['pen']-steal['pen'].shift(1).fillna(0)
    steal['date'] = steal.index
    steal = steal.reset_index(drop=True)
    steal = steal[['date','apple','pen']]
    return steal
#steal.head()

In [12]:
def create_pivot_table(piv_t,df_sell, steal):
    piv_t['state'] = 'MS'
    piv_t.index.name = 'year'
    piv_t.drop(['year'], axis = 1, inplace = True)
    piv_t['apple_sold'] = df_sell['apple']
    piv_t['pen_sold'] = df_sell['pen']
    piv_t['apple_stollen'] = steal['apple']
    piv_t['pen_stollen'] = steal['pen']   
    return piv_t
#piv_t.head()

In [13]:
def upd_piv_tab(piv_t,df_sell,steal):
    piv_t['apple_sold'] += df_sell['apple']
    piv_t['pen_sold'] += df_sell['pen']
    piv_t['apple_stollen']+= steal['apple']
    piv_t['pen_stollen'] += steal['pen']
    return piv_t
#piv_t.head()

In [14]:
file_names = ['MS-b1', 'MS-b2', 'MS-m1', 'MS-m2', 'MS-s1', 'MS-s2', 'MS-s3', 'MS-s4', 'MS-s5']
folder = './output/'

for i in file_names:
    df_sell = pd.read_csv(i+'-sell.csv')
    df_supply = pd.read_csv(i+'-supply.csv')
    df_inv = pd.read_csv(i+'-inventory.csv')
    
    df_sell = sell_dataframe_change(df_sell)
    df_daily_inv = daily_inv( df_sell, df_supply)
    df_daily_inv.to_csv(folder+i+'-output-daily_inv.csv', index = False)
    
    steal = month_steal(df_inv, df_daily_inv)
    steal.to_csv(folder+i+'-steal.csv', index = False)
 
    df_sell.index= df_sell.date
    df_sell.drop(['date'],axis = 1, inplace=True)
    df_sell.index= pd.to_datetime(df_sell.index)
    df_sell=df_sell.resample('Y').sum()
    steal.index=steal.date
    steal.drop(['date'],axis = 1, inplace=True)
    steal.index= pd.to_datetime(steal.index)    
    steal= steal.resample('Y').sum()

    if i == file_names[0]:
        piv_t = pd.DataFrame(columns = ['year', 'state', 'apple_sold', 
                                        'apple_stollen', 'pen_sold', 
                                        'pen_stollen'],index = steal.index)
        piv_t = create_pivot_table(piv_t,df_sell, steal)
    else:
        piv_t = upd_piv_tab(piv_t,df_sell, steal)
piv_t.to_csv(folder+'pivot_table.csv', index = True)
piv_t

,state,apple_sold,apple_stollen,pen_sold,pen_stollen
year,,,,,
2006-12-31,MS,2152006,418.0,155633,461.0
2007-12-31,MS,2150384,377.0,154730,346.0
2008-12-31,MS,2163559,383.0,154597,382.0
2009-12-31,MS,2152502,433.0,155409,454.0
2010-12-31,MS,2149787,418.0,155523,441.0
2011-12-31,MS,2154860,436.0,154158,452.0
2012-12-31,MS,2160040,381.0,155798,421.0
2013-12-31,MS,2157901,361.0,154496,444.0
2014-12-31,MS,2153434,433.0,154687,441.0
